In [1]:
from IPython import get_ipython


def type_of_script():
    """
    Detects and returns the type of python kernel
    :return: string 'jupyter' or 'ipython' or 'terminal'
    """
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return 'jupyter'
        if 'terminal' in ipy_str:
            return 'ipython'
    except:
        return 'terminal'


if type_of_script() == 'jupyter':
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm
    os.nice(19)
    
import matplotlib.pyplot as plt # type: module
import matplotlib.ticker as ticker
from matplotlib import colormaps
from matplotlib.colors import Normalize
import matplotlib.gridspec as gridspec

import numpy as np
import os, glob, atexit, signal
import time
import warnings

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=0
# from rur.sci.kinematics import f_getpot

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in, fancy_axis, circle
from importlib import reload
from copy import deepcopy
from multiprocessing import Pool, shared_memory, Value

from common_func import *

In [2]:
mode1 = 'nh'
database1 = f"/home/jeon/MissingSat/database/{mode1}"
iout1 = 1026
repo1, rurmode1, dp1 = mode2repo(mode1)
snap1 = uri.RamsesSnapshot(repo1, iout1, mode=rurmode1)
snap1s = uri.TimeSeries(snap1)
snap1s.read_iout_avail()
nout1 = snap1s.iout_avail['iout']; nout=nout1[nout1 <= iout1]
gal1s = uhmi.HaloMaker.load(snap1, galaxy=True, double_precision=dp1)
hal1s = uhmi.HaloMaker.load(snap1, galaxy=False, double_precision=dp1)

mode2 = 'nh2'
database2 = f"/home/jeon/MissingSat/database/{mode2}"
iout2 = 797
repo2, rurmode2, dp2 = mode2repo(mode2)
snap2 = uri.RamsesSnapshot(repo2, iout2, mode=rurmode2)
snap2s = uri.TimeSeries(snap2)
snap2s.read_iout_avail()
nout2 = snap2s.iout_avail['iout']; nout=nout2[nout2 <= iout2]
gal2s = uhmi.HaloMaker.load(snap2, galaxy=True, double_precision=dp2)
hal2s = uhmi.HaloMaker.load(snap2, galaxy=False, double_precision=dp2)



LG1 = pklload(f"{database1}/LocalGroup.pickle")
allsubs1 = None
states1 = None
for key in LG1.keys():
    subs = LG1[key]['subs']
    real = LG1[key]['real']
    dink = real[real['state']=='dink']['hid']
    ind = isin(subs['id'], dink)
    subs['dink'][ind] = True
    subs['dink'][~ind] = False
    state = np.zeros(len(subs), dtype='<U7')
    state[ind] = 'dink'
    state[~ind] = 'pair'
    upair = real[real['state']=='upair']['hid']
    ind = isin(subs['id'], upair)
    state[ind] = 'upair'

    allsubs1 = subs if allsubs1 is None else np.hstack((allsubs1, subs))
    states1 = state if states1 is None else np.hstack((states1, state))
argsort = np.argsort(allsubs1['id'])
allsubs1 = allsubs1[argsort]
states1 = states1[argsort]

LG2 = pklload(f"{database2}/LocalGroup.pickle")
allsubs2 = None
states2 = None
for key in LG2.keys():
    subs = LG2[key]['subs']
    real = LG2[key]['real']
    dink = real[real['state']=='dink']['hid']
    ind = isin(subs['id'], dink)
    subs['dink'][ind] = True
    subs['dink'][~ind] = False
    state = np.zeros(len(subs), dtype='<U7')
    state[ind] = 'dink'
    state[~ind] = 'pair'
    upair = real[real['state']=='upair']['hid']
    ind = isin(subs['id'], upair)
    state[ind] = 'upair'

    allsubs2 = subs if allsubs2 is None else np.hstack((allsubs2, subs))
    states2 = state if states2 is None else np.hstack((states2, state))
argsort = np.argsort(allsubs2['id'])
allsubs2 = allsubs2[argsort]
states2 = states2[argsort]

In [3]:
stree1 = pklload(f"{database1}/stable_tree.pickle")
stree2 = pklload(f"{database2}/stable_tree.pickle")

In [18]:
fnames = os.listdir(f"{database1}/stable_prog/")
fnames = [fname for fname in fnames if(fname.startswith("subhalos"))]
fnames.sort()

In [26]:
nstar = pklload(f"{database1}/nstar.pickle")

In [28]:
nstar.shape

(865,)

In [29]:
len(nout1)

865

In [ ]:
def _insitu_SF(ith, maxbstar, maxnstar, address, shape, dtype):
    global ihals, isnap, reft, refn

    exist = shared_memory.SharedMemory(name=address)
    results = np.ndarray(shape=shape, dtype=dtype, buffer=exist.buf)

    ihal = ihals[ith]
    ibox = np.array([[ihal['x']-ihal['r'], ihal['x']+ihal['r']],
                        [ihal['y']-ihal['r'], ihal['y']+ihal['r']],
                        [ihal['z']-ihal['r'], ihal['z']+ihal['r']]])

    if(isnap.mode == 'nh'):
        star = isnap.get_part_instant(box=ibox, nthread=1, target_fields=['x','y','z','m','epoch','id'], pname='star')
    else:
        star = isnap.get_part_instant(box=ibox, nthread=1, target_fields=['x','y','z','m','family','id'], pname='star')
    newstar = star[(np.abs(star['id']) > maxbstar)&(np.abs(star['id']) <= maxnstar)]
    if(len(newstar)>0):
        newstar = cut_sphere(newstar, ihal['x'], ihal['y'], ihal['z'], ihal['r'])
        if(len(newstar)>0):
            results['insitu'][ith] = np.sum(newstar['m','Msol'])
            newstar = cut_sphere(newstar, ihal['x'], ihal['y'], ihal['z'], ihal['rvir'])
            if(len(newstar)>0):
                results['insitu_vir'][ith] = np.sum(newstar['m','Msol'])
    del newstar, star

    refn.value += 1
    if(refn.value%500==0)&(refn.value>0):
        print(f" > {refn.value}/{len(results)} {time.time()-reft.value:.2f} sec (ETA: {(len(results)-refn.value)*(time.time()-reft.value)/refn.value/60:.2f} min)")

In [32]:
snap1.star

array([ True])

In [20]:
ncpu = 48
nstar = pklload(f"{database1}/nstar.pickle")
if(not os.path.isdir(f"{database1}/stable_prog/props")):
    os.mkdir(f"{database1}/stable_prog/props")
for fname in fnames:
    iout = int(fname[-12:-7])
    isnap = snap1s.get_snap(iout)
    if(not isnap.star[0]): continue
    istep = np.where(nout1==iout)[0][0]
    maxbstar = nstar[istep-1]
    maxnstar = nstar[istep]
    ihals = pklload(f"{database1}/stable_prog/{fname}")[0]
    cpulist = isnap.get_halos_cpulist(ihals, 1, radius_name='r', nthread=ncpu)
    uri.timer.verbose=1
    if(isnap.mode=='nh'):
        isnap.get_part(pname='star', nthread=ncpu, target_fields=['x','y','z','m','epoch','id'], exact_box=False, domain_slicing=True, cpulist=cpulist)
    else:
        isnap.get_part(pname='star', nthread=ncpu, target_fields=['x','y','z','m','family','id'], exact_box=False, domain_slicing=True, cpulist=cpulist)
    reft = Value('f', 0); reft.value = time.time()
    refn = Value('i', 0)
    uri.timer.verbose=0
    print(f"[IOUT {iout:05d}]")
    newdtype = np.dtype( [('lastid', '<i4'),('insitu', '<f8'), ('insitu_vir', '<f8')] )
    results = np.zeros(len(ihals), dtype=newdtype)
    memory = shared_memory.SharedMemory(create=True, size=results.nbytes)
    results = np.ndarray(results.shape, dtype=newdtype, buffer=memory.buf)
    results['lastid'] = ihals['lastid']
    with Pool(processes=ncpu) as pool:
        async_result = [pool.apply_async(_insitu_SF, (ith, maxbstar, maxnstar, memory.name, results.shape, newdtype)) for ith in range(nsub)]
        # iterobj = tqdm(async_result, total=len(async_result), desc=f"IOUT{iout:05d} ")# if(uri.timer.verbose>=1) else async_result
        iterobj = async_result
        for r in iterobj:
            r.get()
    pklsave(results, f"{database1}/stable_prog/props/insitu_{iout:05d}.pickle")
    try:
        memory.close()
        memory.unlink()
    except: pass


01026


In [21]:
tmp = pklload(f"{database1}/stable_prog/{fname}")

In [25]:
tmp[0].dtype

dtype([('nparts', '<i4'), ('id', '<i4'), ('timestep', '<i4'), ('level', '<i4'), ('host', '<i4'), ('hostsub', '<i4'), ('nbsub', '<i4'), ('nextsub', '<i4'), ('aexp', '<f8'), ('m', '<f8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<f8'), ('Lx', '<f8'), ('Ly', '<f8'), ('Lz', '<f8'), ('r', '<f8'), ('a', '<f8'), ('b', '<f8'), ('c', '<f8'), ('ek', '<f8'), ('ep', '<f8'), ('et', '<f8'), ('spin', '<f8'), ('sigma', '<f8'), ('rvir', '<f8'), ('mvir', '<f8'), ('tvir', '<f8'), ('cvel', '<f8'), ('rho0', '<f8'), ('rc', '<f8'), ('mcontam', '<f8'), ('r10_mem', '<f8'), ('r50_mem', '<f8'), ('r90_mem', '<f8'), ('r10_vir', '<f8'), ('r50_vir', '<f8'), ('r90_vir', '<f8'), ('r10_max', '<f8'), ('r50_max', '<f8'), ('r90_max', '<f8'), ('sub', '<i8'), ('dink', '?'), ('mdm', '<f8'), ('mstar', '<f8'), ('mcold', '<f8'), ('mcell', '<f8'), ('Host', '<i4'), ('r200kpc', '<f8'), ('m200', '<f8'), ('r200', '<f8'), ('mdm_vir', '<f8'), ('mstar_vir', '<f8'), ('mcell_vir', '<f8'), ('mcold_vi

In [7]:
stree1.keys()

dict_keys([693, 1309, 1310, 1324, 1359, 1389, 1604, 1659, 1676, 1738, 1754, 1755, 1756, 1771, 1773, 1775, 1800, 1906, 1939, 1940, 1941, 2078, 2082, 2099, 2119, 3428, 3430, 3467, 3472, 3484, 3486, 3787, 3797, 3800, 3802, 3831, 3885, 4128, 4797, 4979, 5008, 5010, 5063, 5066, 5069, 5466, 5483, 5488, 5489, 5490, 5493, 5514, 5515, 5522, 5537, 5600, 5607, 5613, 5618, 5630, 5661, 5662, 5665, 5673, 5712, 5737, 5740, 5752, 5754, 5807, 5823, 5825, 5929, 5932, 5936, 5942, 6016, 6370, 6594, 6598, 6610, 8041, 8053, 8054, 8055, 8062, 8157, 8251, 8263, 8274, 8307, 8342, 8343, 8351, 8353, 9434, 9498, 9513, 9515, 9516, 9878, 9885, 11192, 11938, 12016, 13433, 13479, 14049, 14055, 14117, 14221, 14239, 14253, 14259, 14319, 14322, 14376, 14385, 14389, 14393, 14399, 14406, 14408, 14422, 14521, 14525, 14526, 14540, 14542, 14567, 14595, 14618, 18157, 18162, 18170, 18301, 18304, 20513, 20516, 20520, 20521, 20522, 20523, 20524, 20527, 20528, 20532, 20533, 20536, 20538, 20541, 20543, 20546, 20547, 20548, 20549, 